In [1]:
import numpy as np
import face_recognition
import cv2
import pandas as pd 
import os 

In [15]:
## collect the images that used as a database 
## use face detection for seperating each face in the image 
## collect the new face and compare to get the resault 

In [18]:
face_seperation=cv2.CascadeClassifier(r'F:\Computer vision course\addititional resource\OpenCv-Haarcascade-XML-files-master\haarcascade_frontalface_default.xml')

In [3]:
def detect_faces_location(img) :
    faces_location=face_seperation.detectMultiScale(img,1.3,5)
    faces=[]
    for x,y,w,h in faces_location :
        faces.append(img[y:y+h,x:x+w])
    for i in range(len(faces)) :
        faces[i]=cv2.cvtColor(faces[i],cv2.COLOR_BGR2RGB)
    return faces
    

In [4]:
def store_face(faces) :
    face_landmark=[]
    if faces !=[] :
        for face in faces :
            face_landmark.append(face_recognition.face_encodings(face)[0])      
    return face_landmark 

In [5]:

file=r'E:\older_level_compo\face_project'
def read_database(file) :
    land_mark=[]
    list_names=[]
    for files in os.listdir(file) :
        list_names.append(files)
        for img in os.listdir(file+"\\"+files) :
            img_down=cv2.imread(file+"\\"+files+"\\"+img)
            land_mark.append(store_face(detect_faces_location(img_down)))
    return land_mark,list_names
            
            
            
        

In [6]:
land_mark,list_names=read_database(file)
##now we load the database we need know to check the performance 

In [7]:
def check_attendance(img) :
    faces=[]
    prevent_double=[]
    check=[]
    faces=detect_faces_location(img)
    land_marks=store_face(faces)
    for i in range(len(land_marks)) :
        for j in range(len(land_mark)) :
            if face_recognition.compare_faces(land_mark[j],land_marks[i])[0]:
                prevent_double.append(list_names[j])
    prevent_double=set(prevent_double)
    for list_ in prevent_double :
        check.append(list_)
    return check

In [8]:
###create sheet for attendence 
def create_attendence(list_day,list_names,dataframe) :
    day_atten=np.zeros(len(list_names))
    for i in range (len(list_names)) :
        for j in range(len(list_day)) :
            if list_names[i]==list_day[j] :
                day_atten[i]=1
    day_atten=pd.Series(day_atten,index=dataframe.columns)
    dataframe=dataframe.append(day_atten,ignore_index=True)
    return dataframe

In [9]:
##frist_day
img_check_loc=r'E:\older_level_compo\OLD_PHONE\Telegram\IMG_20211114_171825_493.jpg'
img_check=cv2.imread(img_check_loc)
day_1=check_attendance(img_check)
day_1

['osama', 'atef', 'zyad']

In [10]:
##second_day
img_check2_loc=r'E:\older_level_compo\OLD_PHONE\Telegram\IMG_20211118_053302_568.jpg'
img_check2=cv2.imread(img_check2_loc)
day_2=check_attendance(img_check2)
day_2

['atef', 'ahmed']

In [11]:
##third_day
img_check3_loc=r'E:\older_level_compo\check\zizo2.jpg'
img_check3=cv2.imread(img_check3_loc)
day_3=check_attendance(img_check3)
day_3

['abd_elaziz']

In [12]:
sheet=pd.DataFrame(columns=list_names)
day_1_sheet=create_attendence(day_1,list_names,sheet)
day_2_sheet=create_attendence(day_2,list_names,day_1_sheet)
day_3_sheet=create_attendence(day_3,list_names,day_2_sheet)
day_3_sheet.to_csv(r'E:\older_level_compo\attendence_file.csv')

C:\Users\RAM\AppData\Local\Temp\ipykernel_12596\2141312125.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe=dataframe.append(day_atten,ignore_index=True)


In [15]:
##now we check this in the webcamb 
cap=cv2.VideoCapture(0)
attendence=[]
while(True) :
    ret,frame=cap.read()
    if ret ==False :
        break 
    cv2.imshow('frame',frame)
    frame=cv2.blur(frame,(5,5))
    if cv2.waitKey(1)==ord('t') :
        attendence=check_attendance(frame)
    write_img=np.zeros((512,512,3))
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(write_img,f'{attendence}',(100,100),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2,cv2.LINE_AA)
    cv2.imshow('attendence',write_img)
    if cv2.waitKey(1)==ord('q') :
        break
cap.release()
cv2.destroyAllWindows()
sheet_camera=create_attendence(attendence,list_names,sheet)
sheet_camera


C:\Users\RAM\AppData\Local\Temp\ipykernel_12596\2141312125.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe=dataframe.append(day_atten,ignore_index=True)


,abd_elaziz,ahmed,atef,osama,zyad
0,0.0,1.0,0.0,0.0,0.0
